In [1]:
import warnings

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from tobler.dasymetric import masked_area_interpolate
from tobler.model import glm
from tobler.area_weighted import area_interpolate
from openpyxl import load_workbook

from data_preprocessing import read_table, merge_str, load_trim_attr

warnings.filterwarnings('ignore')

In [2]:
# Function to get the index of columns

def get_column_number(file_path, column_name):
    
    df = read_table(file_path)
    
    i=0
    for column in df:
        if str(column) == column_name:
            break
        i = i+ 1
        
    return i

In [3]:
def save_geo_table(df, target_shp, table_name, path=''):
    
    polygons_column = df.geometry.name
    interpolated_attr = df.merge(target_shp, on=polygons_column, how='left')
    interpolated_attr.drop([polygons_column], axis=1, inplace = True)
    interpolated_attr.to_csv(path + table_name + '.csv', index=False)
    
    return interpolated_attr

In [4]:
def areal_interpolation(source_shp, target_shp, attribute_path, filter_path,
                       attribute_name, columns_of_interest, pivot_col, shp_pivot_col,
                       crs=7899, sheet_number=0, header_row=0, code_column=0):
    
    filtered_attr = load_trim_attr(attribute_path, 
                                   columns_of_interest, 
                                   filter_path, 
                                   attribute_name, 
                                   sheet_number, 
                                   header_row, 
                                   code = code_column)
  
    source_shp_attr = merge_str(source_shp, 
                                filtered_attr, 
                                pivot_col, 
                                pivot_col, 
                                how='right')
    
    source_shp_attr[attribute_name] = pd.to_numeric(source_shp_attr[attribute_name])
    
    interpolated_attr = area_interpolate(source_df=source_shp_attr, 
                                         target_df=target_shp,
                                         intensive_variables=[attribute_name])
    return interpolated_attr

In [5]:
def interpolate_db(index, source_shp_path, target_shp_path, filter_path, columns, year, pivot_col, 
                   shp_pivot_col, code_column=0, crs = 7899):
    
    #load shapefiles
    source_shp = gpd.read_file(source_shp_path).to_crs(crs)
    target_shp = gpd.read_file(target_shp_path).to_crs(crs)

    #unify pivot columns' name
    source_shp.rename(columns={shp_pivot_col: pivot_col}, inplace=True)

    counter = 0    
    for row in index[index.columns[columns]].iterrows():
        
        #get params for interpolation
        feature_name = row[1][0]
        feature_file = row[1][1]
        if year == '2006':
            
            attribute_path = 'Data/Demographic_Features/2006/CD_Vic_' + str(feature_file) +'.csv'
            feature_column =row[1][2]
            
        elif year == '2011':
            
            attribute_path = 'Data/Demographic_Features/2011/2011Census_' + str(feature_file) + \
                             '_VIC_SA1_short.csv'
            feature_column =row[1][0]
            
        attribute_name = feature_name
        columns_of_interest = [0,get_column_number(attribute_path, feature_column)]

        #interpolate
        result = areal_interpolation(source_shp, 
                                     target_shp, 
                                     attribute_path, 
                                     filter_path, 
                                     attribute_name, 
                                     columns_of_interest, 
                                     pivot_col, 
                                     shp_pivot_col, 
                                     code_column=code_column)
        
        #Store results
        if counter == 0:
            all_interp = result
        else:
            all_interp[attribute_name] = result[attribute_name]
        counter = counter + 1

    #save file    
    target_shp = gpd.read_file(target_shp_path).to_crs(crs)
    save_geo_table(all_interp,target_shp, 'all_features_inter_' + year)
    
    return all_interp

In [6]:
#Load list with location od source files

feature_index = read_table('Data/Demographic_Features/index_reduced.xlsx')

# 2006

In [ ]:
source_shp_path = 'Data/Shapefiles/un-trimmed/CCD_2006_Victoria/CD06aVIC.shp'
target_shp_path = 'Data/Shapefiles/Great_Melbourne/SA1_2016_GM/SA1_2016_GM.shp'
filter_path = 'Data/Geographical_Units/Great_Melbourne_Units/CCD_2006_GM.csv'
columns = [[3,5,6]]
year= '2006'
pivot_col = 'CD_CODE06'
shp_pivot_col = 'CD_CODE06'
code_column = 0
crs = 7899

demo_ft_2006 = interpolate_db(feature_index, 
                              source_shp_path, 
                              target_shp_path, 
                              filter_path, 
                              columns,
                              year, 
                              pivot_col, 
                              shp_pivot_col, 
                              code_column, 
                              crs)

# 2011

In [200]:
source_shp_path = 'Data/Shapefiles/un-trimmed/SA1_2011_Australia/SA1_2011_AUST.shp'
target_shp_path = 'Data/Shapefiles/Great_Melbourne/SA1_2016_GM/SA1_2016_GM.shp'
filter_path = 'Data/Geographical_Units/Great_Melbourne_Units/SA1_2011_GM.csv'
columns = [[7,9,10]]
year = '2011'
pivot_col = 'SA1_7DIGITCODE_2011'
shp_pivot_col = 'SA1_7DIG11'
code_column = 1
crs = 7899

dummy_2011 = interpolate_db(feature_index, 
                            source_shp_path, 
                            target_shp_path, 
                            filter_path, 
                            columns,
                            year, 
                            pivot_col, 
                            shp_pivot_col, 
                            code_column, 
                            crs)